In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn

import vgg_spiking_bntt as snn_models_bntt

DATA_PATH='/tmp/data/cifar10'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_args = {'num_cls': 100, 'timesteps': 20}
net = snn_models_bntt.SNN_VGG9_BNTT(**model_args).cuda()

trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset_train = datasets.CIFAR10(DATA_PATH, train=True, download=True, transform=trans_cifar)
dataset_test = datasets.CIFAR10(DATA_PATH, train=False, download=True, transform=trans_cifar)


BATCH_SIZE = 32

trainloader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

>>>>>>>>>>>>>>>>>>> VGG 9 >>>>>>>>>>>>>>>>>>>>>>
***** time step per batchnorm
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Files already downloaded and verified
Files already downloaded and verified


In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001, weight_decay = 0.0001, amsgrad = True)

batch_loss = []
for batch_idx, (images, labels) in enumerate(trainloader):
    images, labels = images.to(DEVICE), labels.to(DEVICE)
    net.zero_grad()
    log_probs = net(images)
    loss = loss_func(log_probs, labels)
    loss.backward()
    optimizer.step()
    # if batch_idx % 10 == 0:
    #     print('Update Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
    #         iter, batch_idx * len(images), len(trainloader.dataset),
    #                 100. * batch_idx / len(trainloader), loss.item()))
    # batch_loss.append(loss.item())
    # if (batch_idx + 1) % BATCH_SIZE == 0:
    #     thresholds = []
    #     for value in net.module.threshold.values():
    #         thresholds = thresholds + [round(value.item(), 2)]
    #     print('Epoch: {}, batch {}, threshold {}, leak {}, timesteps {}'.format(iter, batch_idx + 1, thresholds, net.module.leak.item(), net.module.timesteps))